# NN to train and classify Devanagari Handwritten Character Dataset digits

In [4]:
from scipy.misc import imread
import numpy as np
import pandas as pd
import os
root = 'tmp/DevanagariHandwrittenCharacterDataset'

In [198]:
def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    One-hot encoding converts categorical labels to binary values
    """
    y_one_hot = np.zeros((num_classes))
    y_one_hot[y] = 1
    return y_one_hot

In [199]:
def getDataset(path):
    final_path = root+path
    final_data_images = []
    final_data_labels = []
    for directory, subdirectories, files in os.walk(final_path):
        for dr in subdirectories:
            if "digit" in str(dr):
                label = int(dr.split('_')[1])
                dr_path = final_path+"/"+dr
                for d,s,f in os.walk(dr_path):
                    for i in f:
                        image_data = imread(dr_path+"/"+i).flatten()
                        image_data = image_data.reshape(image_data.shape[0])
                        final_data_images.append(image_data)
                        final_data_labels.append(one_hot(label))
    return np.array(final_data_images),np.array(final_data_labels)

In [200]:
def get_train_data():
    train_data,train_labels = getDataset("/train/")
    return train_data,train_labels

In [201]:
def get_test_data():
    test_data,test_labels = getDataset("/test/")
    return test_data,test_labels

In [202]:
train_data,train_labels = get_train_data()
test_data,test_labels = get_test_data()

In [220]:
num_examples = train_data.shape[0]

In [226]:
def get_next_train_batch(batch_size, current_index):
    if(current_index >= len(train_data)):
        current_index = 0
    X_batch,y_batch = train_data[current_index:(current_index+batch_size)], train_labels[current_index:(current_index+batch_size)]
    return X_batch, y_batch, (current_index+batch_size)

### Now that the data preparation is done, lets train the neural network

In [151]:
import tensorflow as tf;

In [152]:
#number of nodes in hidden layer 1, hl2, and hl3
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

# number of output classes = [0,1,2,3,4,5,6,7,8,9]
n_classes = 10

#due to RAM limit, its best to use batches for input processing
batch_size = 100

In [215]:
x = tf.placeholder('float',[None,1024])
y = tf.placeholder('float',[None,10])

In [217]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([1024,n_nodes_hl1])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                     'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    # z = (input_data*weights) + biases
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']),hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output_layer['weights']),output_layer['biases'])
    
    return output

In [235]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
    #default learning rate = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    n_epochs = 50
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(n_epochs):
            epoch_loss = 0
            current_index = 0
            for _ in range(int(num_examples/batch_size)):
                epoch_x,epoch_y,current_index = get_next_train_batch(batch_size, current_index)
                _,c = sess.run([optimizer,cost], feed_dict = {x:epoch_x,y:epoch_y})
                epoch_loss += c
                
            print('epoch', epoch, 'completed out of ', n_epochs, 'loss',epoch_loss)
            if(epoch_loss == 0.0):
                break
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy', accuracy.eval({x:test_data,y:test_labels}))
        

In [236]:
train_neural_network(x)

epoch 0 completed out of  50 loss 2586271742.732422
epoch 1 completed out of  50 loss 537688947.125
epoch 2 completed out of  50 loss 314669534.5566406
epoch 3 completed out of  50 loss 194708601.13232422
epoch 4 completed out of  50 loss 130557671.64398193
epoch 5 completed out of  50 loss 97107493.21679688
epoch 6 completed out of  50 loss 74342444.4621582
epoch 7 completed out of  50 loss 58310928.73693848
epoch 8 completed out of  50 loss 43121814.39984131
epoch 9 completed out of  50 loss 29151392.747680664
epoch 10 completed out of  50 loss 18992226.0732193
epoch 11 completed out of  50 loss 12205553.91241455
epoch 12 completed out of  50 loss 7739739.420471191
epoch 13 completed out of  50 loss 4194705.902893066
epoch 14 completed out of  50 loss 2168271.201789856
epoch 15 completed out of  50 loss 866038.9546356201
epoch 16 completed out of  50 loss 401898.6431121826
epoch 17 completed out of  50 loss 214864.42293071747
epoch 18 completed out of  50 loss 94488.9820022583
epoch 